# Title
[]()

In [2]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
# from datetime import datetime
# sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")

# From `data_functions.py`

In [2]:
import boto3
from boto3.dynamodb.conditions import Key
import json
from datetime import datetime, timezone
import sys
import os

def query_dynamodb_table(table_name, value, key='SessionId', region_name="us-west-2"):
    """
    Query a Dynamodb table and print the results.
    Parameters:
        table_name (str): The name of the DynamoDB table.
        value (str): The value of the key.
        key (str): The name of the key.
        region_name (str): The name of the AWS region.

    Returns:
        dict: The results of the query. The 'Items' key contains the table record.       
    From 2023-10-22 notebook.
    """
    dynamodb = boto3.resource('dynamodb', region_name=region_name)
    table = dynamodb.Table(table_name)
    response = table.query(
        KeyConditionExpression=Key(key).eq(value)
    )
    return response

# Payload

In [3]:
event = {
  "body": {
    "type": "InboundMessage",
    "locationId": "QSZ6lvarPNxtmTFl84W8",
    "attachments": [],
    "body": "This is another test inbound message",
    "contactId": "test",
    "contentType": "text/plain",
    "conversationId": "fcanlLgpbQgQhderivVs",
    "dateAdded": "2023-11-10T12:26:02.000Z",
    "direction": "inbound",
    "messageType": "SMS",
    "userId": "80pzXdC3MCqKRpzrteuj"
  }
}
payload = event["body"]

# Iteration 1 

In [5]:
message_events = ['InboundMessage', 'OutboundMessage', 'NoteCreate']
contact_id_key = 'contactId' if payload['type'] in message_events else 'id'
contact_data = query_dynamodb_table(
    'SessionTable', payload[contact_id_key], key='SessionId'
    )

In [6]:
contact_data

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': 'MRGSD8EQ2SJUUE7JO93SRR6LOJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 11 Nov 2023 20:21:42 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '39',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'MRGSD8EQ2SJUUE7JO93SRR6LOJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3413411624'},
  'RetryAttempts': 0}}

In [8]:
if contact_data['Items']:
    print(contact_data['Items'])
else:
    print('No contact data found')

No contact data found


# `load_txt` function

In [5]:
def load_txt(filename, filepath, encoding='utf-8'):
    """
    Load a text file as a string using the specified file path copied from Windows file explorer.
    Backslashes in the file path will be converted to forward slashes.

    Arguments:
    - filepath (raw string): Use the format r'<path>'.
    - filename (string).

    Returns: string object.
    """
    filename = f'{filepath}/'.replace('\\','/') + filename
    with open(filename, 'r', encoding=encoding) as file:
        text = file.read()
    return text

filename = 'new_contact_messages_2023-11-10_0953_trimmed2023-11-11_1140.txt'
filepath = r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\GHL-chat\data\chat_examples'

examples = load_txt(filename, filepath)
print(examples)

Contact ID: 0X4K1LEBnAt7eX9EC3nC{'OutboundMessage': 'Hi Sejil, Thanks for registering your interest in Strength and Movement Lab, Alexandria.\n\nWhat are you trying to do fitness wise?\n\nThanks\n\nCharlie'}
	{'OutboundMessage': "Hey, it's Charlie from Strength and Movement Lab here.\n\nJust following up on your enquiry again to see if you may be interested in our 7 FREE day small group trial (1 Intro PT session and access to any of our 40 weekly small group sessions). Alternatively we offer a 1-1 trial too.\n\nIf interested let me know your availability to swing by for your intro session so we can chat more about your training and injury history, goals, go through some basic movements and get you ready for the rest of your trial sessions?\n\nThanks"}
	{'OutboundMessage': 'Fantastic, we’ll lock in 6am next Wednesday 13th with me! \n\nWe’re located at 92 Mcevoy St. if you’re driving, there is parking right outside, or on both sidestreets that surround us! \n\nThere’s nothing essential t

# Load markdown

In [8]:
filepath = r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\GHL-chat\data\rag_docs'
filename = 'SAM_Lab_doc.md'
print(load_txt(filename, filepath))

# About Strength & Movement Lab Alexandria
## Contact
Address: 92 McEvoy St, Alexandria  NSW 2015
Phone: 1300472652
https://www.strengthandmovementlab.com.au/

## Schedule
Monday 5:15am–7:30pm
Tuesday 5:15am–7:30pm
Wednesday 5:15am–7:30pm
Thursday 5:15am–7:30pm
Friday 5:15am–6:45pm
Saturday 8:00am–12:30pm
Sunday – Closed

# What to know for your free trial session
Parking is available outside or on both surrounding side streets.
You can even park for free in the car park next to Tuga Pastries!

## Preparing for the session
Complete the pre-exercise questionnaire:
https://api.leadconnectorhq.com/widget/form/52ykF6nV5R1cEyyPzs1b

## Items to bring
There's nothing essential to bring, but some water and a towel is always handy to have!

## What to expect
* The free intro session is show you the gym, and go through some basic movements to ensure you are moving safely and effectively.
* You can then have a free 7 day trial or 14 day trial to our small group training for $59... both include u

# Chatbot: No need to do RAG for initial iteration

In [12]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from SAM_Lab import *

ModuleNotFoundError: No module named 'scripts.SAM_Lab'

In [13]:

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage

from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

def create_system_message(
        business_name, business_dict, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    instructions = load_txt(business_dict[business_name][0], prompts_filepath)
    examples = load_txt(business_dict[business_name][1], examples_filepath)
    document = load_txt(business_dict[business_name][2], doc_filepath)

    system_message = f"""{instructions}

    **Examples**

    {examples}

    **Relevant documentation**

    {document}

    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{question}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

# def create_chatbot(verbose=True, streamlit=False):

#     llm = ChatOpenAI(
#         temperature = 0,
#         openai_organization=os.environ['openai_organization'],
#         openai_api_key=os.environ['openai_api_key'],
#         )
#     message_history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="1")
#     memory = ConversationBufferMemory(
#         memory_key="ChatHistory", chat_memory=message_history, return_messages=True
#     )
#     system_message = SystemMessage(
#         content=("""
#             You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
#             You ask enough follow up questions as needed to provide the most relevant answer. 
#             Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
#             Recycle BC is the main resource for recycling information. 
#             Respond to the resident's query, which are delimited by triple backticks: ```{question}```\n\n
#             If relevant, indicate the source website from which you are basing your response.
#             """
#         ),
#         input_variables=['question']
#     )
    
#     prompt = OpenAIFunctionsAgent.create_prompt(
#         system_message=system_message,
#         extra_prompt_messages=[
#             MessagesPlaceholder(variable_name='chat_history')
#             ]
#     )

#     agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
#     agent_executor = AgentExecutor(
#         agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
#         )
#     agent_info = {
#         'agent': agent,
#         'agent_executor': agent_executor,
#         'memory': memory,
#         'chat_history': [],
#         'msgs': msgs if streamlit == True else None
#     }
#     return agent_info

# def chat_with_chatbot(user_input, agent_info, streamlit=False):
#     start_time = time()
#     print(f'Chat history length: {len(agent_info["chat_history"])}')
#     if streamlit == False:
#         chat_history = agent_info['chat_history']
#     else: 
#         if type(agent_info['msgs'].messages) == list:
#             chat_history = agent_info['msgs'].messages
#         else:
#             chat_history = [agent_info['msgs'].messages]
#     result = agent_info['agent_executor']({
#         "input": user_input,
#         "chat_history": chat_history
#         })
#     agent_info['chat_history'].append(result['chat_history'])
#     print(f'Response time: {time() - start_time} seconds')
    
#     return result

business_dict = {
    'SAM_Lab': (
        'SAM_Lab.txt', # business-specific system message
        'new_contact_messages_2023-11-10_0953_trimmed2023-11-11_1140.txt', # Examples 
        'SAM_Lab_doc.md' # RAG doc
        )
}

print(create_system_message('SAM_Lab', business_dict))

You are a helpful assistant working at a gym that offers small group fitness classes and personal 
training. You reach out to new leads via SMS message with the goal of getting them to sign up as clients.

Here is the process you go through:
1. Ask about their fitness goals.
2. Once the lead responds, schedule an introductory session at the gym.
3. Once the introductory session is scheduled, have them complete the pre-exercise questionnaire
at this link: https://api.leadconnectorhq.com/widget/form/52ykF6nV5R1cEyyPzs1b
4. At the introductory session, you offer them to sign up for one of the following: 
    * a free 7-day trial for small group training.
    * a 14-day trial for small group training for $59
    * the first 2 personal training sessions for $99

Additionally, you do the following:
* Where relevant, you retrieve the relevant information from documentation to write your response.
* Follow up with leads if they don't respond after a few days.
* Answer queries from leads.
* If 

# *End of Page*